In [1]:
import pandas as pd

In [11]:
dt=pd.read_csv("bcancer.csv")
dt.drop('Unnamed: 0',axis=1,inplace=True)
dt=dt.replace('?','15')
dt['g']=dt['g'].astype(dtype='i')
x=dt.drop(['a','class'],axis=1)
y=dt['class']
from sklearn.model_selection import train_test_split
x_tr,x_ts,y_tr,y_ts=train_test_split(x,y,test_size=0.2)

In [12]:
from sklearn.naive_bayes import GaussianNB
algo=GaussianNB()
algo.fit(x_tr,y_tr)
algo.score(x_ts,y_ts)

0.9857142857142858

In [13]:
from sklearn.neighbors import KNeighborsClassifier
algo1=KNeighborsClassifier()
algo1.fit(x_tr,y_tr)
algo1.score(x_ts,y_ts)

0.9785714285714285

In [17]:
class_prob=dict(y_tr.value_counts(normalize=True))

In [24]:
d=dict.fromkeys(y_tr)
for key in d:
    d[key]=dict.fromkeys(range(x_tr.shape[1]))
    for col in d[key]:
        d[key][col]=[]
d

{2: {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []},
 4: {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}}

In [25]:
for ind in range(x_tr.shape[0]):
    for col in range(x_tr.shape[1]):
        d[y_tr.iloc[ind]][col].append(x_tr.iloc[ind].values[col])

In [26]:
for key in d:
    for col in d[key]:
        d[key][col]=dict(pd.Series(d[key][col]).value_counts(normalize=True))

In [27]:
d

{2: {0: {1: 0.32054794520547947,
   2: 0.10410958904109589,
   3: 0.19726027397260273,
   4: 0.14246575342465753,
   5: 0.18356164383561643,
   6: 0.038356164383561646,
   7: 0.0027397260273972603,
   8: 0.010958904109589041},
  1: {1: 0.8136986301369863,
   2: 0.08767123287671233,
   3: 0.06575342465753424,
   4: 0.021917808219178082,
   6: 0.0027397260273972603,
   7: 0.0027397260273972603,
   8: 0.0027397260273972603,
   9: 0.0027397260273972603},
  2: {1: 0.7534246575342466,
   2: 0.12054794520547946,
   3: 0.07945205479452055,
   4: 0.024657534246575342,
   5: 0.00821917808219178,
   6: 0.005479452054794521,
   7: 0.005479452054794521,
   8: 0.0027397260273972603},
  3: {1: 0.810958904109589,
   2: 0.07671232876712329,
   3: 0.07397260273972603,
   4: 0.0136986301369863,
   5: 0.010958904109589041,
   6: 0.010958904109589041,
   10: 0.0027397260273972603},
  4: {1: 0.10684931506849316,
   2: 0.7835616438356164,
   3: 0.06301369863013699,
   4: 0.01643835616438356,
   5: 0.01095890

In [47]:
def predict(x,cls,d):
    cl=cls.copy()
    for ind in cl:
        for col in range(len(x)):
            cl[ind]*=d[ind][col].get(x[col],0)
    return max(cl,key=cl.get)


In [49]:
c=0
for i in range(x_ts.shape[0]):
    if predict(x_ts.iloc[i],class_prob,d)==y_ts.iloc[i]:
        c+=1
c/x_ts.shape[0]

0.9928571428571429

In [50]:
class GNB:
    def fit(s,x_tr,y_tr):
        s.cls=dict(y_tr.value_counts(normalize=True))
        s.d=dict.fromkeys(y_tr)
        for key in d:
            s.d[key]=dict.fromkeys(range(x_tr.shape[1]))
            for col in d[key]:
                s.d[key][col]=[]
        for ind in range(x_tr.shape[0]):
            for col in range(x_tr.shape[1]):
                s.d[y_tr.iloc[ind]][col].append(x_tr.iloc[ind].values[col])
        for key in d:
            for col in d[key]:
                s.d[key][col]=dict(pd.Series(s.d[key][col]).value_counts(normalize=True))
    def predict(s,x):
        cl=s.cls.copy()
        for ind in cl:
            for col in range(len(x)):
                cl[ind]*=s.d[ind][col].get(x[col],0)
        return max(cl,key=cl.get)
    def score(s,x_ts,y_ts):
        c=0
        for i in range(x_ts.shape[0]):
            if s.predict(x_ts.iloc[i])==y_ts.iloc[i]:
                c+=1
        return c/x_ts.shape[0]

In [51]:
alg=GNB()
alg.fit(x_tr,y_tr)
alg.score(x_ts,y_ts)

0.9928571428571429